# Configuracion de grafica a usar

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# lA ID de la GPU a usar, puede ser desde 0 hasta las N GPU's. Si es -1 significa que es en la CPU
os.environ["CUDA_VISIBLE_DEVICES"]="-1";

# Importacion de librerias

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from IPython.display import display, clear_output

%matplotlib inline
import sys
sys.path.append('../')
from Datasets_utils.DatasetsLoader import VideoDataGenerator

# Configuraciones para Tensorflow y Keras

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [ ]:
tf.debugging.set_log_device_placement(False)

In [ ]:
#Comprobar que estoy ejecutandome en modo eagerly
tf.executing_eagerly()

# Carga de Datos

In [ ]:
root_path = "/home/jefelitman/DataSets/ucf101/split_1"
root_path

In [ ]:
batch = 30
original_size = [171,128]
size = [112,112]
frames = 16
canales = 3

In [ ]:
dataset = VideoDataGenerator(directory_path = root_path, 
                             batch_size = batch, 
                             original_frame_size = original_size, 
                             frame_size=size, 
                             video_frames = frames, 
                             temporal_crop = ('random', 4), 
                             frame_crop = ('random', 2), 
                             shuffle = True, 
                             conserve_original = True)

# Red Neuronal LTC

### Construccion del modelo

In [ ]:
#Entrada de la red neuronal
video_shape = tuple([frames]+size+[canales])
dropout = 0.5
entrenamiento = True
lr = 1e-3

In [ ]:
entrada = keras.Input(shape=video_shape,
                     batch_size=batch,
                     name="Input_video")
#Conv1
x = keras.layers.Conv3D(filters=64, kernel_size=3, padding="same", activation="relu", use_bias=False)(entrada)
x = keras.layers.MaxPool3D(pool_size=(1,2,2),strides=(1,2,2))(x)

#Conv2
x = keras.layers.Conv3D(filters=128, kernel_size=3, padding="same", activation="relu", use_bias=False)(x)
x = keras.layers.MaxPool3D(pool_size=(2,2,2),strides=(2,2,2))(x)

#Conv3
x = keras.layers.Conv3D(filters=256, kernel_size=3, padding="same", activation="relu", use_bias=False)(x)
x = keras.layers.MaxPool3D(pool_size=(2,2,2),strides=(2,2,2))(x)

#Conv4
x = keras.layers.Conv3D(filters=256, kernel_size=3, padding="same", activation="relu", use_bias=False)(x)
x = keras.layers.MaxPool3D(pool_size=(2,2,2),strides=(2,2,2))(x)

#Conv5
x = keras.layers.Conv3D(filters=256, kernel_size=3, padding="same", activation="relu", use_bias=False)(x)
x = keras.layers.MaxPool3D(pool_size=(2,2,2),strides=(1,1,1))(x)

#fc6
x = tf.reshape(tensor=x, shape=[batch,-1], name="Aplanado")
x = keras.layers.Dense(2048, activation="relu")(x)
x = keras.layers.Dropout(dropout)(x, training=entrenamiento)

#fc7
x = keras.layers.Dense(2048, activation="relu")(x)
x = keras.layers.Dropout(dropout)(x, training=entrenamiento)

#fc8
salidas= keras.layers.Dense(len(dataset.to_class), activation="softmax")(x)

ltc = keras.Model(entrada, salidas, name="LTC")
ltc.summary()

In [ ]:
#Optimizador del modelo
optimizador = keras.optimizers.SGD(learning_rate=lr)
perdida = keras.losses.SparseCategoricalCrossentropy()

In [ ]:
#keras.utils.plot_model(ltc, 'LTC.png', show_shapes=True)

In [ ]:
ltc = keras.models.load_model('/home/jefelitman/Saved_Models/LTC_112x112x16_rgb_batch30/ltc.h5'')

In [ ]:
ltc.summary()

### Entrenamiento de la red

In [ ]:
def precision(y, y_predicho):
    clases = tf.argmax(y_predicho, axis=1)
    comparaciones = tf.equal(y, clases)
    return tf.reduce_mean(tf.cast(comparaciones, tf.int32))

In [ ]:
step = 1
train_epoch = 1
train_loss = []
train_accuracy = []
maximo = -1
step_max = 0

In [ ]:
while True:
    if step % 1000 == 0:
        ltc.save('/home/jefelitman/Saved_Models/LTC_112x112x16_rgb_batch30/ltc.h5')
        print('Modelo salvado en la iteracion: {i}'.format(i=step))
        
    if step == 80000 or step == 125000:
        lr = lr * 0.1
        optimizador = keras.optimizers.SGD(learning_rate=lr)
    
    if dataset.train_batch_index == dataset.train_batches:
        train_epoch += 1
        
    clear_output(wait=True)
    print("Step: ",step)
    print("Train Epoch: ",train_epoch, " Train batch: ",dataset.train_batch_index+1,"/",dataset.train_batches)
    
    batch, labels = dataset.get_next_train_batch(canales)
    
    with tf.GradientTape() as tape:
        predicciones = ltc(batch)
        valor_perdida = perdida(labels, predicciones)
    
    train_loss.append(valor_perdida)    
    train_accuracy.append(precision(labels, predicciones))
    
    if train_accuracy[-1] > maximo:
        maximo = train_accuracy[-1]
        step_max = step
        
    print("Train_Loss: ",train_loss[-1].numpy()," Train_Acuraccy: ",train_accuracy[-1].numpy())
    print("Tain_max_Acuraccy: {m} at the step: {s}".format(m=maximo, s=step_max))
        
    grads = tape.gradient(valor_perdida, ltc.trainable_weights)
    
    optimizador.apply_gradients(zip(grads, ltc.trainable_weights))
    
    step += 1